### INSTALL REQUIRED LIBRARIES AND PACKAGES

In [11]:
# Install LangChain integration for Gemini models
# Install LangChain community
!pip install langchain-google-genai google-generativeai langchain_community

In [12]:
!pip install -U langchain-ollama


# CREATE SQL DATABASE

In [15]:
# import required libraries
import pandas as pd
from sqlalchemy import create_engine


# Load diabetes dataset
# pandas is used to read structured tabular data
df=pd.read_csv("diabetes.csv")

# Normalize column names
# Rename columns for SQL friendliness
df.columns = df.columns.str.lower()

# create SQLite database engine
# database will be saved as diabetes.db
engine = create_engine("sqlite:///diabetes.db")

# Store the dataset as a SQL table
df.to_sql("patients",engine,if_exists="replace",index=False)

# ensure that table is recreated only if it already exists

print("Database created successfully")


Database created successfully


### Connect SQLite medical database to LLM

In [16]:
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_ollama import OllamaLLM

# Database
engine = create_engine("sqlite:///diabetes.db")

db = SQLDatabase(
    engine,
    include_tables=["patients"]
)



### LOCAL LLM (OLLAMA)

In [ ]:
# Initialize the LLM- Ollama
llm = OllamaLLM(
    model="mistral",
    temperature=0
)


### Create sql agent

In [18]:
# SQL Agent 
agent = create_sql_agent(
    llm=llm,
    db=db,
    verbose=True,
    max_iterations=5,
    agent_executor_kwargs={
        # handle parsing errors prevetns crashes
        "handle_parsing_errors": True
    }
)

In [ ]:
# Query
# agent converts the question into a SQL query 
response=agent.invoke(
    {"input":"Show the patient record with the hghest glucose level."}

# print the final result
print(response["output"])




> Entering new SQL Agent Executor chain...
 Action: sql_db_list_tables
Action Input: ""patients Now that I know there is a table named "patients", I should query its schema to find the relevant columns.

Action: sql_db_schema "patients"
Action Input: ""sql_db_schema "patients" is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].